In [ ]:
import pandas as pd
from tqdm import tqdm
from cpi import inflate
import numpy as np
import pycountry
import ast
import re
from nameparser import HumanName
from racebert import RaceBERT

# Enable progress bar with tqdm
tqdm.pandas()

# Load movie metadata
movies = pd.read_csv("../../data/MovieSummaries/movie.metadata.tsv", sep='\t')

# Load character metadata
characters = pd.read_csv("../../data/MovieSummaries/character.metadata.tsv", sep='\t')

# Load plot summaries
summaries = pd.read_csv("../../data/MovieSummaries/plot_summaries.txt", sep='\t')

In [ ]:
def extract_movie_year(movies_df):
    """
    Extract just the year from the 'Release date' column in the movies DataFrame.
    """
    year_pattern = re.compile(r'(\d{4})')
    movies_df['Release date'] = movies_df['Release date'].progress_apply(
        lambda x: int(year_pattern.search(str(x)).group(0)) if pd.notnull(x) and year_pattern.search(str(x)) else None
    )
    movies_df['Release date'] = movies_df['Release date'].astype('Int64')
    return movies_df

def extract_languages(movies_df):
    """
    Extract language names from the structured format in the 'Languages' column 
    and remove the word 'Language' if present.
    """
    def extract_language_values(language_str):
        try:
            # Handle empty or null values
            if not isinstance(language_str, str) or not language_str:
                return []
            
            # Convert string representation to dictionary
            language_dict = ast.literal_eval(language_str)
            
            # Get values and clean them
            languages = language_dict.values()
            cleaned_languages = [
                lang.replace(' Language', '').replace('Language ', '').strip()
                for lang in languages
            ]
            
            return cleaned_languages
            
        except (ValueError, SyntaxError):
            return []
    
    movies_df['Languages'] = movies_df['Languages'].progress_apply(extract_language_values)
    return movies_df

def extract_genres(movies_df):
    """
    Extract genre names from the structured format in the 'Genres' column.
    """
    def extract_genre_values(genre_str):
        try:
            # Handle empty or null values
            if not isinstance(genre_str, str) or not genre_str:
                return []
            
            # Convert string representation to dictionary
            genre_dict = ast.literal_eval(genre_str)
            return list(genre_dict.values())
            
        except (ValueError, SyntaxError):
            return []
    
    movies_df['Genres'] = movies_df['Genres'].progress_apply(extract_genre_values)
    return movies_df

def extract_countries(movies_df):
    """
    Extract and harmonize country names from the structured format in the 'Countries' column.
    """
    def process_countries(country_str):
        try:
            # Handle empty or null values
            if not isinstance(country_str, str) or not country_str:
                return []
            
            # Convert string representation to dictionary and process countries
            country_dict = ast.literal_eval(country_str)
            return [
                pycountry.countries.search_fuzzy(country)[0].name 
                for country in country_dict.values()
                if pycountry.countries.search_fuzzy(country)
            ]
            
        except (ValueError, SyntaxError, LookupError):
            return []
    
    movies_df['Countries'] = movies_df['Countries'].progress_apply(process_countries)
    return movies_df

def adjust_revenue(movies_df):
    """
    Adjust box office revenue for inflation to 2023 dollars.
    Uses CPI data for years >= 1913 and applies a fixed annual inflation rate for earlier years.
    Converts final values to Int64 (long integer).
    
    Parameters:
    movies_df (pd.DataFrame): DataFrame containing 'Box office revenue' and 'Release date' columns
    
    Returns:
    pd.DataFrame: DataFrame with adjusted revenue converted to Int64
    """
    # Define constants
    TARGET_YEAR = 2023
    EARLIEST_CPI_YEAR = 1913
    HISTORICAL_INFLATION_RATE = 0.023  # 2.3% average annual inflation before 1913
    
    def adjust_single_revenue(row):
        try:
            # Skip if no revenue or release date
            if pd.isnull(row['Box office revenue']) or pd.isnull(row['Release date']):
                return None
                
            revenue = float(row['Box office revenue'])
            year = int(row['Release date'])
            
            # Handle current or future years
            if year >= TARGET_YEAR:
                return revenue
                
            # Use CPI library for years 1913 and later
            if year >= EARLIEST_CPI_YEAR:
                return inflate(revenue, year, TARGET_YEAR)
                
            # For years before 1913, compound a fixed inflation rate
            years_before_cpi = EARLIEST_CPI_YEAR - year
            historical_factor = (1 + HISTORICAL_INFLATION_RATE) ** years_before_cpi
            
            # First adjust to 1913 using historical rate, then use CPI to adjust to target year
            revenue_1913 = revenue * historical_factor
            return inflate(revenue_1913, EARLIEST_CPI_YEAR, TARGET_YEAR)
            
        except (ValueError, TypeError):
            return None
    
    # Apply the adjustment with progress bar
    movies_df['Box office revenue'] = movies_df.progress_apply(adjust_single_revenue, axis=1)
    
    # Convert to Int64 (long integer), rounding the float values first
    movies_df['Box office revenue'] = movies_df['Box office revenue'].round().astype('Int64')
    
    return movies_df

def clean_movies(movies_df):
    """
    Clean the movie dataset, including inflation adjustment for revenue with progress tracking.
    Parameters:
    movies_df (pd.DataFrame): The movies DataFrame to be cleaned
    
    Returns:
    pd.DataFrame: The cleaned movies DataFrame
    """
    # Extract just the year from the release date
    movies_df = extract_movie_year(movies_df)
    
    # Extract language strings
    movies_df = extract_languages(movies_df)
    
    # Extract country strings
    movies_df = extract_countries(movies_df)
    
    # Extract genre strings
    movies_df = extract_genres(movies_df)
    
    # Adjust box office revenue for inflation
    movies_df = adjust_revenue(movies_df)
    
    # Convert runtime to integer
    movies_df['Runtime'] = movies_df['Runtime'].fillna(0).astype('int64')
    
    return movies_df
    

In [ ]:
# Usage example
movies = clean_movies(movies)

# Save cleaned data
movies.to_csv("../../data/clean/movie_metadata_cleaned.csv", index=False)

In [ ]:
characters.head()

In [2]:
def extract_actor_age_at_release(characters_df):
    """
    Compute actor ages after dates have been cleaned.
    Validates against possible ages based on year difference.
    """
    def calculate_age(row):
        if pd.notnull(row['Release date']) and pd.notnull(row['Actor date of birth']):
            # Calculate two possible ages
            age1 = row['Release date'] - row['Actor date of birth']
            age2 = row['Release date'] - row['Actor date of birth'] - 1
            
            # If existing age matches one of possible ages, keep it
            if pd.notnull(row['Actor age at release']) and row['Actor age at release'] in [age1, age2]:
                return row['Actor age at release']
            
            # Otherwise return the higher age
            return age1
        return None
    
    characters_df['Actor age at release'] = characters_df.progress_apply(calculate_age, axis=1)
    characters_df['Actor age at release'] = characters_df['Actor age at release'].astype('Int64')
    
    return characters_df

def extract_character_year(characters_df):
    """
    Extract just the year from the 'Release date' column in the characters DataFrame.
    """
    year_pattern = re.compile(r'(\d{4})')
    characters_df['Release date'] = characters_df['Release date'].progress_apply(
        lambda x: int(year_pattern.search(str(x)).group(0)) if pd.notnull(x) and year_pattern.search(str(x)) else None
    )
    characters_df['Release date'] = characters_df['Release date'].astype('Int64')
    return characters_df

def extract_character_year_of_birth(characters_df):
    """
    Extract just the year from the 'Release date' column in the characters DataFrame.
    """
    year_pattern = re.compile(r'(\d{4})')
    characters_df['Actor date of birth'] = characters_df['Actor date of birth'].progress_apply(
        lambda x: int(year_pattern.search(str(x)).group(0)) if pd.notnull(x) and year_pattern.search(str(x)) else None
    )
    characters_df['Actor date of birth'] = characters_df['Actor date of birth'].astype('Int64')
    return characters_df

def extract_ethnicity(characters_df):
    """
    Predict actor ethnicity using RaceBERT model for records with valid names.
    Returns list of ethnicities with confidence scores above threshold.
    Skips prediction for empty or invalid names.
    """
    CONFIDENCE_THRESHOLD = 0.5  # Minimum confidence to include an ethnicity
    
    # Initialize RaceBERT model
    model = RaceBERT()
    
    def predict_ethnicity(name):
        """Predict ethnicity for a single name"""
        # Check for empty, null, or whitespace-only names
        if pd.isnull(name) or not str(name).strip():
            return []
            
        try:
            # Get prediction from RaceBERT
            prediction = model.predict_ethnicity(name)
            
            # Convert string prediction to dictionary if it's a string
            if isinstance(prediction, str):
                prediction = ast.literal_eval(prediction)
            
            # If single prediction, wrap in list
            if isinstance(prediction, dict):
                prediction = [prediction]
            
            # Filter predictions above threshold and extract labels
            ethnicities = [pred['label'] for pred in prediction 
                         if pred.get('score', 0) >= CONFIDENCE_THRESHOLD]
            
            return ethnicities if ethnicities else []
            
        except Exception as e:
            print(f"Error predicting ethnicity for name '{name}': {str(e)}")
            return []
            
    characters_df['Actor ethnicity'] = characters_df['Actor name'].progress_apply(predict_ethnicity)
    
    return characters_df


def clean_characters(characters_df):
    """
    Clean the character dataset
    """
    
    # Extract just the year from the release date
    characters_df = extract_character_year(characters_df)
    
    # Extrqct date of birth
    characters_df = extract_character_year_of_birth(characters_df)
    
    # Recompute the Actor age at release just in case and convert to Int64
    characters_df = extract_actor_age_at_release(characters_df)
    
    # Extract actor ethnicity
    characters_df = extract_ethnicity(characters_df)
    
    return characters_df
    

In [3]:
characters = clean_characters(characters)

# Save cleaned data
characters.to_csv("../../data/clean/character_metadata_cleaned.csv", index=False)

100%|██████████| 450669/450669 [4:15:14<00:00, 29.43it/s]  
